In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
chrome_options.add_argument("--headless") 

# Set up the Chrome WebDriver
service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the specific oddschecker page
url = "https://www.oddschecker.com/politics/us-politics/us-presidential-election/winner"
driver.get(url)

# Wait for the page to load
time.sleep(10)

# Get the page source
page_source = driver.page_source

# Close the browser
driver.quit()

In [8]:
# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table or relevant parts
odds_table = soup.find('tbody', id='t1')

# Extract each row and the data within
odds_data = []
bookmakers_set = set()

for row in odds_table.find_all('tr'):
    market_name = row.find('a', class_='popup').text.strip()  # Extract the party name
    odds_dict = {'Market': market_name}
    
    # Find all td elements with odds information
    for td in row.find_all('td', class_=lambda x: x and ('o' in x.split() or 'bs' in x.split())):
        bookmaker = td.get('data-bk')  # Extract the bookmaker name
        decimal_odds = td.get('data-odig')  # Extract the decimal odds value
        if bookmaker and decimal_odds:  # Only add if both are present
            odds_dict[bookmaker] = float(decimal_odds)  # Convert odds to float
            bookmakers_set.add(bookmaker)
    
    odds_data.append(odds_dict)

# Create a DataFrame with all bookmakers as columns
df = pd.DataFrame(odds_data).set_index('Market')

# Ensure all bookmakers are columns, even if some are missing in certain rows
df = df.reindex(columns=sorted(bookmakers_set))

print(df)

                       AKB     B3      BF     BY     CE     DP    EE      FB  \
Market                                                                         
Kamala Harris         1.91    1.8    1.94    1.8    1.8    1.8   1.8    1.80   
Donald Trump          2.05    2.0    2.10    2.0    2.0    2.0   2.0    1.91   
Robert Kennedy Jr.  151.00   67.0  245.00  101.0   67.0  101.0  67.0  126.00   
J D Vance           151.00   81.0     NaN    NaN  101.0    NaN   NaN     NaN   
Nikki Haley         151.00   67.0  294.00  101.0   67.0  101.0  51.0  151.00   
...                    ...    ...     ...    ...    ...    ...   ...     ...   
Dwayne Johnson         NaN    NaN  980.00    NaN    NaN    NaN   NaN     NaN   
Bernie Sanders         NaN  201.0  980.00    NaN  401.0    NaN   NaN     NaN   
Tim Scott              NaN    NaN  980.00    NaN    NaN    NaN   NaN     NaN   
Jill Stein             NaN    NaN     NaN    NaN    NaN    NaN   NaN     NaN   
Cornel West            NaN    NaN  980.0